In [4]:
import pandas as pd
import openai
from dotenv import load_dotenv
import json
import os

In [5]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [6]:
#load data
df = pd.read_csv('end_res.csv')


In [9]:
def sim_gen(abs):
    sys_prompt = "Given a medical abstract, generate a simplified version that anyone can understand."
    
    response = openai.ChatCompletion.create(
        model='gpt-4', 
        messages=[
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": "Medical abstract:\n\n{}\n".format(abs)}
        ]
    )
    
    return response['choices'][0]['message']['content']

In [10]:
df['simplification'] = df.apply(lambda x: sim_gen(x.Abstract), axis=1)

In [12]:
df[['Abstract', 'simplification']]

,Abstract,simplification
0,ABSTRACT.BACKGROUND: Antibiotics are overused ...,Abstract in simple terms:\n\nBackground: Antib...
1,ABSTRACT.BACKGROUND: Minimal hepatic encephalo...,Simplified Version:\n\nThis study was about a ...
2,ABSTRACT.OBJECTIVE: To determine whether glarg...,Simplified Version:\n\nThis research aimed to ...
3,ABSTRACT.BACKGROUND: ACE inhibition results in...,Summary for non-scientific audience:\n\nThis r...
4,ABSTRACT.BACKGROUND:: Oral isotretinoin is hig...,Simplified version:\n\nThis study was conducte...


In [13]:
new_data = []
for ind, row in df.iterrows():
    entry = {
        "source": row['Abstract'],
        "target": row['simplification'],
        "edits": [
            {
                "category": "hol_eval",
                "id": 0
            },
            {
                "category": "final_q",
                "id": 1
            },
            {
                "category": "pico_pop",
                "id": 2
            },
            {
                "category": "pico_inter",
                "id": 3
            },
            {
                "category": "pico_comp",
                "id": 4
            },
            {
                "category": "pico_out",
                "id": 5
            },
            {
                "category": "results",
                "id": 6,
                "input_idx": [
                    [
                        row['Evidence Start'],
                        row['Evidence End']
                    ]
                ]
            }
        ]   
    }

In [14]:
with open('results_test.json', 'w') as d:
    json.dump(new_data, d, indent=4)